# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "cis"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,brca_Database_ID,lscc_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
ADAMDEC1,NP_055294.1|NP_001138743.1,NP_055294.1|NP_001138744.1,NP_055294.1|NP_001138743.1,NP_001138744,8.742524e-01,5.845904e-01,7.556452e-01,8.531975e-01,1.421191e-01,0.369805,-0.126551,-0.165344,-0.088317,0.115958,-1.167415,-0.379442
AGPAT5,NP_060831.2,NP_060831.2,NP_060831.2,NP_060831,1.388856e-03,1.167418e-03,1.308774e-03,9.501841e-02,1.900678e-04,0.008375,-0.882857,-0.313480,-0.264665,-0.400303,-0.960898,-0.348131
ANGPT2,NP_001112359.1|NP_001138.1|NP_001112360.1,NP_001138.1|NP_001112360.1|NP_001112359.1,NP_001112359.1|NP_001138.1,NP_001112359,5.362941e-01,NaN,9.157472e-01,7.081900e-01,5.195903e-01,0.369805,0.235629,NaN,0.021822,0.160530,-0.219792,-0.466135
ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,NP_001295082.1|NP_055444.2|NP_001295081.1,NP_001295082.1,NP_001295081,1.357847e-03,5.845904e-01,4.686530e-02,3.689966e-01,2.020514e-03,0.106523,-0.669850,-0.089861,-0.161051,-0.213564,-0.578240,-0.218074
ASAH1,NP_808592.2|NP_004306.3|NP_001350672.1|NP_0011...,NP_004306.3|NP_808592.2,NP_004306.3|NP_808592.2|NP_001350672.1|NP_0011...,NP_004306,5.790637e-02,1.616228e-02,4.255651e-02,8.631828e-01,1.067770e-01,0.054226,-0.624626,-0.271963,-0.228478,0.066940,-0.621295,-0.257519
ATP6V1B2,NP_001684.2,NP_001684.2,NP_001684.2,NP_001684,6.308363e-05,4.828611e-08,1.133689e-02,1.072099e-03,5.005967e-05,0.019213,-0.429505,-0.293465,-0.156175,-0.451716,-0.581165,-0.164293
BIN3,NP_061158.1|NP_001349975.1,NP_061158.1,NP_061158.1|NP_001349975.1,NP_061158,4.170954e-02,8.544040e-02,7.958375e-01,8.268740e-01,3.230197e-01,0.270463,-0.532597,-0.119531,-0.033641,0.056429,-0.206668,-0.124552
BLK,NP_001706.2|NP_001317394.1,NP_001706.2|NP_001317394.1,NaN,NP_001317394,2.375428e-01,NaN,3.324858e-01,NaN,1.299998e-02,0.535973,0.346266,NaN,0.258200,NaN,-2.269184,-0.153329
BMP1,NP_006120.1|NP_001190.1,NP_006120.1|NP_001190.1,NP_006120.1|NP_001190.1,NP_006120,1.657147e-01,NaN,2.568742e-02,2.199438e-01,1.355766e-01,0.126460,-0.489970,NaN,0.412351,-0.667694,-0.706473,-0.317478


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,ADAMDEC1,NP_055294.1|NP_001138744.1,8.742524e-01,-0.126551
1,brca,AGPAT5,NP_060831.2,1.388856e-03,-0.882857
2,brca,ANGPT2,NP_001138.1|NP_001112360.1|NP_001112359.1,5.362941e-01,0.235629
3,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,1.357847e-03,-0.669850
4,brca,ASAH1,NP_004306.3|NP_808592.2,5.790637e-02,-0.624626
5,brca,ATP6V1B2,NP_001684.2,6.308363e-05,-0.429505
6,brca,BIN3,NP_061158.1,4.170954e-02,-0.532597
7,brca,BLK,NP_001706.2|NP_001317394.1,2.375428e-01,0.346266
8,brca,BMP1,NP_006120.1|NP_001190.1,1.657147e-01,-0.489970
9,brca,BNIP3L,NP_004322.1|NP_001317420.1,1.703058e-02,-0.904697


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,AGPAT5,NP_060831.2,1.388856e-03,-0.882857,-1
1,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,1.357847e-03,-0.669850,-1
2,brca,ATP6V1B2,NP_001684.2,6.308363e-05,-0.429505,-1
3,brca,BIN3,NP_061158.1,4.170954e-02,-0.532597,-1
4,brca,BNIP3L,NP_004322.1|NP_001317420.1,1.703058e-02,-0.904697,-1
5,brca,CDCA2,NP_689775.2|NP_001304835.1|NP_001304836.1,1.398641e-02,-0.968210,-1
6,brca,CHMP7,NP_689485.1|NP_001304828.1,2.397773e-09,-0.805922,-1
7,brca,CNOT7,NP_001309022.1|NP_001309020.1|NP_001309023.1|N...,4.423579e-03,-0.380102,-1
8,brca,DOCK5,NP_079216.4,5.837377e-05,-0.763545,-1
9,brca,ERI1,NP_699163.2,3.566806e-06,-0.955015,-1


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
ATP6V1B2,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
CHMP7,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
ERI1,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
KIF13B,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
PPP2R2A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
VPS37A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
XPO7,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
CNOT7,"[brca, colon, hnscc, lscc, luad]",-1,5
DOCK5,"[brca, colon, hnscc, lscc, luad]",-1,5


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

6     7
5    12
4     2
3     8
2     9
1    19
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")

## Info for most common proteins

- [AGPAT5](https://www.uniprot.org/uniprot/Q9NUQ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Converts 1-acyl-sn-glycerol-3-phosphate (lysophosphatidic acid or LPA) into 1,2-diacyl-sn-glycerol-3-phosphate (phosphatidic acid or PA) by incorporating an acyl moiety at the sn-2 position of the glycerol backbone (PubMed:21173190). Acts on LPA containing saturated or unsaturated fatty acids C15:0-C20:4 at the sn-1 position using C18:1-CoA as the acyl donor (PubMed:21173190). Also acts on lysophosphatidylethanolamine using oleoyl-CoA, but not arachidonoyl-CoA, and lysophosphatidylinositol using arachidonoyl-CoA, but not oleoyl-CoA (PubMed:21173190). Activity toward lysophosphatidylglycerol not detectable (PubMed:21173190).
- ATP6V1B2 	(colon, hnscc, lscc, luad, ovarian) 	5
- CCDC25 	(colon, hnscc, lscc, luad, ovarian) 	5
- CHMP7 	(colon, hnscc, lscc, luad, ovarian) 	5
- [ERI1](https://www.uniprot.org/uniprot/Q8IV48)    (colon, hnscc, lscc, luad, ovarian)     5
    - RNA exonuclease that binds to the 3'-end of histone mRNAs and degrades them, suggesting that it plays an essential role in histone mRNA decay after replication. A 2' and 3'-hydroxyl groups at the last nucleotide of the histone 3'-end is required for efficient degradation of RNA substrates. Also able to degrade the 3'-overhangs of short interfering RNAs (siRNAs) in vitro, suggesting a possible role as regulator of RNA interference (RNAi). Requires for binding the 5'-ACCCA-3' sequence present in stem-loop structure. Able to bind other mRNAs. Required for 5.8S rRNA 3'-end processing. Also binds to 5.8s ribosomal RNA. Binds with high affinity to the stem-loop structure of replication-dependent histone pre-mRNAs.
- [PPP2CB](https://www.uniprot.org/uniprot/P62714)  (colon, hnscc, lscc, luad, ovarian)     5
    - PP2A can modulate the activity of phosphorylase B kinase casein kinase 2, mitogen-stimulated S6 kinase, and MAP-2 kinase.
- [PPP2R2A](https://www.uniprot.org/uniprot/P63151)     (colon, hnscc, lscc, luad, ovarian)     5
    - The B regulatory subunit might modulate substrate selectivity and catalytic activity, and also might direct the localization of the catalytic enzyme to a particular subcellular compartment.
- [VPS37A](https://www.uniprot.org/uniprot/Q8NEZ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Component of the ESCRT-I complex, a regulator of vesicular trafficking process. Required for the sorting of endocytic ubiquitinated cargos into multivesicular bodies. May be involved in cell growth and differentiation.
- [XPO7](https://www.uniprot.org/uniprot/Q9UIA9)    (colon, hnscc, lscc, luad, ovarian)  5
    - Mediates the nuclear export of proteins (cargos) with broad substrate specificity. In the nucleus binds cooperatively to its cargo and to the GTPase Ran in its active GTP-bound form. Docking of this trimeric complex to the nuclear pore complex (NPC) is mediated through binding to nucleoporins. Upon transit of a nuclear export complex into the cytoplasm, disassembling of the complex and hydrolysis of Ran-GTP to Ran-GDP (induced by RANBP1 and RANGAP1, respectively) cause release of the cargo from the export receptor. XPO7 then return to the nuclear compartment and mediate another round of transport. The directionality of nuclear export is thought to be conferred by an asymmetric distribution of the GTP- and GDP-bound forms of Ran between the cytoplasm and nucleus.